### Import Modules

In [175]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [190]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [191]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True, # alphaS
        'include_log2' : False
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma-taylor', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
    
    'make_plots' : False,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [192]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.32400012016
Compiling results...


Fit type: ma-taylor (F^2 = FKFpi, bsN = 1)

Fitting to nnlo 
 with volume corrections O(0) 
 chained: False 
Fitted/[FLAG] values at physical point (including SU(2) isospin corrections):

F_K / F_pi = 1.1807(41) [1.1932(19)]   (delta_su2 = -0.00197(22))

  Parameter    Result[0] / Prior[1]    Parameter    Result[0] / Prior[1]
-----------------------------------  -----------------------------------
   A_loga 0               1.1 (4.2)            1               0.0 (5.0)
          1               0.0 (5.0)        A_a 0               0.4 (2.8)
      L_5 0            0.00043 (57)            1               0.0 (5.0)
          1             0.0000 (50)        A_p 0               1.19 (44)
      A_k 0              -0.9 (2.4)            1               0.0 (5.0)
---
boot0 fit results:
Least Square Fit:
  chi2/dof [dof] = 2 [16]    Q = 0.0096    logGBF = 54.984

Settings:
  svdcut/n = 1e-12/

In [ ]:
bootstrapper.order

In [183]:
translator_dict = {
    'xpt' : 'xpt-ratio', 
    'xpt-taylor' : 'xpt',
    'ma' : 'ma-ratio', 
    'ma-taylor' : 'ma',
}

def make_table(bootstrapper):
    name = translator_dict[bootstrapper.fit_type]
    if bootstrapper.order['vol'] > 0:
        name = name + '_FV'

    if bootstrapper.order['include_log']:
        name = name + '_alphaS'

    if bootstrapper.order['include_log2']:
        name = name + '_logSq'
        
    print name
    
make_table(bootstrapper)

ma_alphaS


In [181]:
bootstrapper.order

{'exclude': [],
 'fit': 'nnlo',
 'include_log': True,
 'include_log2': False,
 'vol': 0}

In [193]:
print bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)

1.1819(42)


In [9]:
fit_data['a09m220']['mpi'].shape

(5001L,)

In [289]:
#print bootstrapper.fits[0]

In [275]:
for abbr in bootstrapper.abbrs:
    print abbr, bootstrapper.fit_data[abbr]['alpha_s']

a09m220 0.43356000(43)
a09m310 0.43356000(43)
a09m350 0.43356000(43)
a09m400 0.43356000(43)
a12m130 0.53796000(54)
a12m220 0.53796000(54)
a12m220L 0.53796000(54)
a12m220S 0.53796000(54)
a12m310 0.53796000(54)
a12m350 0.53796000(54)
a12m400 0.53796000(54)
a15m135XL 0.58801000(59)
a15m220 0.58801000(59)
a15m310 0.58801000(59)
a15m350 0.58801000(59)
a15m400 0.58801000(59)


In [274]:
print [bootstrapper.fit_data[abbr]['alpha_s'] for abbr in bootstrapper.abbrs]

[0.43356000(43), 0.43356000(43), 0.43356000(43), 0.43356000(43), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.53796000(54), 0.58801000(59), 0.58801000(59), 0.58801000(59), 0.58801000(59), 0.58801000(59)]


In [ ]:
def test(self):
    p
    

In [248]:
def test(self):
    temp = bootstrapper.fits[0].p
    print gv.corr(temp['mpi'][0], temp['mk'][0])
    
test(bootstrapper)

0.442983188623


In [235]:
bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)

1.1804(57)

In [190]:
def test(self):
    return gv.corr(self.fit_data['a09m220']['mpi'], self.fit_data['a09m220']['mk'])
test(bootstrapper)

0.46065693199088703

In [185]:
def test(self):
    for abbr in self.abbrs:
        print abbr, self.fit_data[abbr]['FK/Fpi'], self.extrapolate_to_ensemble(abbr)

test(bootstrapper)

a09m220 1.1439(18) 1.1447(12)
a09m310 1.1108(16) 1.11160(62)
a09m350 1.0997(10) 1.09822(64)
a09m400 1.08107(94) 1.08085(61)
a12m130 1.1646(26) 1.1628(12)
a12m220 1.1314(23) 1.13444(83)
a12m220L 1.1279(29) 1.13362(82)
a12m220S 1.1389(27) 1.13783(95)
a12m310 1.1042(21) 1.10467(58)
a12m350 1.0888(27) 1.09154(47)
a12m400 1.0745(17) 1.07609(55)
a15m135XL 1.1584(28) 1.1559(15)
a15m220 1.1266(15) 1.12685(97)
a15m310 1.1016(13) 1.10095(56)
a15m350 1.08868(87) 1.08793(55)
a15m400 1.07151(89) 1.07239(61)


### Or make all fits

In [44]:
p_dict = {
    'order' : {
        'vol' : 5, # max 10
        'exclude' : [],
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [45]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Making fits...
100% complete Time (s):  0.463999986649
Compiling results...


../fitter\special_functions.py:10: RuntimeWarning: invalid value encountered in double_scalars
  ysdev = np.abs(ss.kvp(n, gv.mean(g), 1)) * gv.sdev(g)


Done.
Saving...
Done.

Total time (s):  11.7779998779 

xpt nnlo
Making fits...
100% complete Time (s):  0.979000091553
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.881000042 

xpt-taylor nlo
Making fits...
100% complete Time (s):  0.468999862671
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.63800001144 

xpt-taylor nnlo
Making fits...
100% complete Time (s):  0.984000205994
Compiling results...
Done.
Saving...
Done.

Total time (s):  9.89199995995 

ma nlo
Making fits...
100% complete Time (s):  0.991000175476
Compiling results...
Done.
Saving...
Done.

Total time (s):  54.1529998779 

ma nnlo
Making fits...
100% complete Time (s):  2.49399995804
Compiling results...
Done.
Saving...
Done.

Total time (s):  56.1860001087 

ma-taylor nlo
Making fits...
100% complete Time (s):  1.11199998856
Compiling results...
Done.
Saving...
Done.

Total time (s):  49.6940000057 

ma-taylor nnlo
Making fits...
100% complete Time (s):  2.46700000763
Compiling results

In [38]:
#bootstrapper.fits[0].show_plots(view='std')

### Compare with Andre

In [155]:
p_dict = {
    'order' : {
        'fit' : 'nnlo'
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : False,

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
}

In [156]:
results_dict = {}

for fit_type in ['xpt', 'ma', 'xpt-taylor', 'ma-taylor']:#, 'ma-old']:
    results_dict[fit_type] = {}
    for include_log2 in [True, False]:
        results_dict[fit_type][include_log2] = {}
        for include_log in [True, False]:
            results_dict[fit_type][include_log2][include_log] = {}
            for vol in [0, 10]:
                results_dict[fit_type][include_log2][include_log][vol] = {}
                #print fit_type, order_fit

                # Specify fit parameters
                p_dict['order']['include_log'] = include_log
                p_dict['order']['include_log2'] = include_log2
                p_dict['order']['vol'] = vol

                p_dict['fit_type'] = fit_type

                # Do fit

                # Load data
                data_loader = dl.data_loader()
                fit_data = data_loader.get_fit_data()

                # Get prior
                prior = None

                # Make bootstrapper
                bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])


                results_dict[fit_type][include_log2][include_log][vol]['fit'] = bootstrapper.extrapolate_to_phys_point(include_su2_isospin_corrrection=False)
                #results_dict[fit_type][include_log2][include_log][vol]['L5'] = bootstrapper.get_fit_parameters('L_5')
                results_dict[fit_type][include_log2][include_log][vol]['info'] = bootstrapper.get_fit_info()
                #results_dict[fit_type][include_log2][include_log][vol]['fit'] = 
                print 

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.96799993515
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.999000072479
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.982000112534
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.09200000763
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.05800008774
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.10599994659
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.983000040054
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  0.925999879837
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  1.49099993706
Compiling results...

Using default prior.
Making fits...
Done!
100% complete Time (s):  2.

In [171]:
translator_dict = {
    'xpt' : 'xpt-ratio', 
    'xpt-taylor' : 'xpt',
    'ma' : 'ma-ratio', 
    'ma-taylor' : 'ma',
}

In [173]:
#print "name   FK/Fpi   L5   k   p   s"

k = 30 - len("name")
output = "name" +k*' '

for param in ['FK/Fpi', 'L5', 'k', 'p', 's']:
    k = 14 - len(param)
    output += k*" " + str(param)

print output
                        
                        
print "---"

#for fit_type in results_dict.keys():
    #for include_log2 in results_dict[fit_type].keys():
for include_log2 in results_dict['ma'].keys():
    for include_log in results_dict[fit_type][include_log2].keys():
        for vol in results_dict[fit_type][include_log2][include_log].keys():
            for fit_type in ['xpt-taylor', 'ma-taylor']:
                
                name = translator_dict[fit_type]
                if vol > 0:
                    name = name + '_FV'
                    
                if include_log:
                    name = name + '_alphaS'
                
                if include_log2:
                    name = name + '_logSq'
                    
                    
                fit = results_dict[fit_type][include_log2][include_log][vol]['fit']
                L5 = results_dict[fit_type][include_log2][include_log][vol]['info']['L_5']
                Ak = results_dict[fit_type][include_log2][include_log][vol]['info']['A_k']
                Ap = results_dict[fit_type][include_log2][include_log][vol]['info']['A_p']
                Aa = results_dict[fit_type][include_log2][include_log][vol]['info']['A_a']
                
                if ((vol>0), include_log, include_log2) in [(1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]:
                    if len(name) > 0:
                        k = 30 - len(name)
                        output = name +k*' '
                        
                        for param in [fit, L5, Ak, Ap, Aa]:
                            k = 14 - len(str(param))
                            output += k*" " + str(param)
                            
                        print output
                
                    


name                                  FK/Fpi            L5             k             p             s
---
xpt                               1.1809(52)  -0.00035(49)      2.3(2.1)      1.59(39)     -3.32(36)
ma                                1.1811(31)   0.00041(57)     -0.8(2.4)      1.18(44)      1.17(40)
xpt_FV                            1.1818(61)  -0.00059(49)      3.3(2.1)      1.36(39)     -3.85(36)
ma_FV                             1.1810(33)   0.00034(57)     -0.5(2.4)      1.11(44)      0.96(40)
xpt_FV_alphaS                     1.1811(67)  -0.00061(50)      3.4(2.1)      1.35(39)     -3.2(2.8)
ma_FV_alphaS                      1.1816(43)   0.00036(58)     -0.6(2.4)      1.12(44)      0.3(2.8)
xpt_FV_alphaS_logSq               1.1840(62)  -0.00071(50)      1.8(2.1)      1.90(39)     -3.2(2.8)
ma_FV_alphaS_logSq                1.1852(47)  -0.00005(58)     -0.8(2.4)      1.43(44)      0.2(2.8)


In [161]:
bootstrapper.get_fit_info()

{'A_a': 0.96(40),
 'A_k': -0.5(2.4),
 'A_p': 1.11(44),
 'L_5': 0.00034(57),
 'Q': 0.045320451356663731,
 'chi2/df': 1.6667686195604041,
 'delta_su2': -0.00194(21),
 'fit': 1.1798(33),
 'logGBF': 57.919990652456242,
 'name': 'ma-taylor_nnlo',
 'vol corr': 10}

In [137]:
(True, True, True) in [(1, 1, 1), (1, 1, 0), (1, 0, 0), (0, 0, 0)]

True

In [128]:
(True, True) == (True, False)

False

In [75]:
len('xpt-taylor_FV_alphaS_logSq')

26

In [47]:
len('xpt-taylor_FV_alphaS')

20